In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scikitplot as skplt
import seaborn as sns
from scipy import linalg

In [2]:
train = pd.read_csv('train.csv', delimiter=',', header = 0)
test = pd.read_csv('test.csv', delimiter=',', header = 0)
user_features = pd.read_csv('user-features.csv', delimiter = ',', header = 0)
item_features = pd.read_csv('item-features.csv', delimiter=',', header = 0)

In [3]:
def code_mean(data, cat_feature, real_feature):
    return dict(data.groupby(cat_feature)[real_feature].mean()), dict(data.groupby(cat_feature)[real_feature].count())


pred = [76,35,37,22,60,65,80,72,66,200,58, 7,11,40,148,36,21,5,59,87]
odd = [12,42,68,157,175,197,207,55,64,161,167,179,185,187,208,69,152,177,195,29]
item_dict = code_mean(train, 'item_id', 'like')
mean = item_dict[0]
count = item_dict[1]
#odd_keys = []
for key, val in count.items():
    if val >= 15:
        if mean[key] >= 0.5 and key not in pred and key not in odd:
            #odd_keys.append(key)
            print(key, val, mean[key])

2 18 0.5
18 22 0.5
32 42 0.5476190476190477
33 36 0.5277777777777778
67 45 0.5111111111111111
136 15 0.6666666666666666
146 41 0.5365853658536586
147 20 0.75


In [4]:
train = train.loc[:, [col for col in train.columns if col != 'timestamp']]
test = test.loc[:, [col for col in test.columns if col != 'timestamp']]
item_features = item_features.sort_values(by=['item_id'])

In [5]:
train = train[train['item_id'] < 209]
item_features = item_features[item_features['item_id'] < 209]

In [6]:
'''for column in user_matrix.T:
    plt.figure()
    #plt.hist(column, bins = 10)
    sns.kdeplot(column)'''

'for column in user_matrix.T:\n    plt.figure()\n    #plt.hist(column, bins = 10)\n    sns.kdeplot(column)'

In [8]:
'''for column in item_matrix.T:
    plt.figure()
    #plt.hist(column, bins = 10)
    sns.kdeplot(column)'''

'for column in item_matrix.T:\n    plt.figure()\n    #plt.hist(column, bins = 10)\n    sns.kdeplot(column)'

In [10]:
n_users = user_features.shape[0]
n_items = item_features.shape[0]
train_arr = train.values
interactions = np.zeros(n_users*n_items).reshape(n_users, n_items)
for line in train_arr:
    if int(line[2]) == 1:
        interactions[line[0]][line[1]] = 1
    else:
        interactions[line[0]][line[1]] = -1

In [11]:
interactions

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [-1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
user_matrix = user_features.iloc[:,1:].values
item_matrix = item_features.iloc[:,1:].values

In [13]:
user_matrix.shape

(497, 32)

In [14]:
U, s, Vh = linalg.svd(interactions, full_matrices=False)
S = np.diag(s[:32])

In [15]:
'''A1 = (user_matrix-np.min(user_matrix))/(np.max(user_matrix)-np.min(user_matrix))
A2 = (U[:, :32]-np.min(U[:, :32]))/(np.max(U[:, :32])-np.min(U[:, :32]))
A1 = user_matrix / np.max(user_matrix)
A2 = U[:, :32] / np.max(U[:, :32])'''

#A1 - A2

'A1 = (user_matrix-np.min(user_matrix))/(np.max(user_matrix)-np.min(user_matrix))\nA2 = (U[:, :32]-np.min(U[:, :32]))/(np.max(U[:, :32])-np.min(U[:, :32]))\nA1 = user_matrix / np.max(user_matrix)\nA2 = U[:, :32] / np.max(U[:, :32])'

In [13]:
print(U.shape)
print(s.shape)
print(Vh.shape)

(497, 209)
(209,)
(209, 209)


In [16]:
predictions = np.dot(user_matrix, (S @ item_matrix.T))
normalized = (predictions-np.min(predictions))/(np.max(predictions)-np.min(predictions))

In [18]:
test_ids = test['user_id'].tolist()
n_items = item_features.shape[0]

In [22]:
counter = 0
def full_pred(pred, liked):
    global counter
    counter += 1
    j = 0
    liked2 = [59, 148, 200, 2, 18, 26, 51, 56, 61, 110, 126, 150, 19, 49, 83, 77, 73, 142, 193]
    while len(pred) < 20:
        if liked2[j] not in liked:
            pred.append(liked2[j])
        else:
            j+=1
        j+=1
    return pred

a = []
for user_x in test_ids:
    scores = normalized[user_x]
    score_dict = {i: scores[i] for i in range(len(scores))}
    sorted_ = sorted(score_dict.items(), key=lambda item: item[1], reverse = True) 
    pred = [k for k, v in sorted_]
    liked = [] 
    for i in range(len(interactions[user_x])):
        if int(interactions[user_x][i]) == -1 and i in pred:
            pred.remove(i)
        if int(interactions[user_x][i]) == 1:
            if i in pred:
                pred.remove(i)
            liked.append(i)
    if len(pred) < 20:       
        #print(user_x)
        pred = full_pred(pred, liked)
    
    
    a.append([user_x] + pred[:20])
#print(counter)

In [21]:
a = np.array(a)
cols = ['user_id'] + [str(i) for i in range(20)]
ans = pd.DataFrame(data = a, columns = cols)
ans.head()

,user_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,166,76,35,37,22,60,65,80,72,66,...,58,7,40,11,148,36,21,5,59,146
1,26,76,35,37,22,60,65,80,72,66,...,58,7,40,11,148,36,21,5,59,146
2,41,76,35,37,22,60,65,80,72,66,...,58,40,11,148,36,21,5,59,146,87
3,286,76,35,37,22,60,65,80,72,66,...,58,7,11,148,36,21,5,59,87,146
4,108,76,35,37,22,60,80,72,66,200,...,7,40,11,148,36,21,5,59,146,87


In [23]:
ans.to_csv('ans_.csv', index = False)